## Домашнее задание к занятию "Классификация: Логистическая регрессия и SVM"

Имеются данные adult.csv (см. в материалах к занятию).
Целевой переменной является уровень дохода income (крайний правый столбец).
Описание признаков можно найти по ссылке www.cs.toronto.edu...etail.html
Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.
Готовый ноутбук выложить на гитхаб и прислать ссылку.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv( 'adult.csv' )
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [4]:
print('Всего строк в наборе {}'.format( len( data ) ))

Всего строк в наборе 48842


1. Изначально Всего 48842 записей;
2. null (пропусков) значений нет;
3. Но есть непонятные значения '?' в разных столбцах. Будем удалять эти строки;
4. По очереди проверяю все столбцы и вношу изменения если нужны

In [5]:
# Возраст. Здесь все хорошо
data['age'].unique()

array([25, 38, 28, 44, 18, 34, 29, 63, 24, 55, 65, 36, 26, 58, 48, 43, 20,
       37, 40, 72, 45, 22, 23, 54, 32, 46, 56, 17, 39, 52, 21, 42, 33, 30,
       47, 41, 19, 69, 50, 31, 59, 49, 51, 27, 57, 61, 64, 79, 73, 53, 77,
       80, 62, 35, 68, 66, 75, 60, 67, 71, 70, 90, 81, 74, 78, 82, 83, 85,
       76, 84, 89, 88, 87, 86], dtype=int64)

In [6]:
data['workclass'].unique()

array(['Private', 'Local-gov', '?', 'Self-emp-not-inc', 'Federal-gov',
       'State-gov', 'Self-emp-inc', 'Without-pay', 'Never-worked'],
      dtype=object)

In [7]:
data['workclass'].value_counts()

Private             33906
Self-emp-not-inc     3862
Local-gov            3136
?                    2799
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: workclass, dtype: int64

In [8]:
# здесь пристутсвует '?' в количестве 2799 из 48842. Буду удалять эти строки
data = data.loc[data['workclass'] != '?']
data['workclass'].value_counts()

Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: workclass, dtype: int64

In [9]:
# примерная оценка количества людей, которое представляет каждая строка данных
# data['fnlwgt'].unique()
# data['fnlwgt'].value_counts().sort_index(ascending=True)
# data['fnlwgt'] != '?'


# Пропускаю этот столбец...

In [10]:
# длительность обучения. Также все хорошо
data['education'].unique()

# Все ок

array(['11th', 'HS-grad', 'Assoc-acdm', 'Some-college', '10th',
       'Prof-school', '7th-8th', 'Bachelors', 'Masters', 'Doctorate',
       '5th-6th', 'Assoc-voc', '9th', '12th', '1st-4th', 'Preschool'],
      dtype=object)

In [11]:
# длительность обучения. Также все хорошо
data['educational-num'].unique()

array([ 7,  9, 12, 10,  6, 15,  4, 13, 14, 16,  3, 11,  5,  8,  2,  1],
      dtype=int64)

In [12]:
# Семейное положение. Также все хорошо
data['marital-status'].unique()

array(['Never-married', 'Married-civ-spouse', 'Widowed', 'Separated',
       'Divorced', 'Married-spouse-absent', 'Married-AF-spouse'],
      dtype=object)

In [13]:
# Занятость. Также все хорошо
data['occupation'].unique()

array(['Machine-op-inspct', 'Farming-fishing', 'Protective-serv',
       'Other-service', 'Prof-specialty', 'Craft-repair', 'Adm-clerical',
       'Exec-managerial', 'Tech-support', 'Sales', 'Priv-house-serv',
       'Transport-moving', 'Handlers-cleaners', 'Armed-Forces', '?'],
      dtype=object)

In [14]:
# Есть пропущенные значения: '?'
# смотрим сколько таких строк:
data['occupation'].value_counts()

Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
?                      10
Name: occupation, dtype: int64

In [15]:
# Таких 10 строк. Удаляю их:
data = data.loc[data['occupation'] != '?']
data['occupation'].value_counts()

Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: occupation, dtype: int64

In [16]:
# Отношения. Также все хорошо
data['relationship'].unique()

array(['Own-child', 'Husband', 'Not-in-family', 'Unmarried', 'Wife',
       'Other-relative'], dtype=object)

In [17]:
# Раса. Также все хорошо
data['race'].unique()

array(['Black', 'White', 'Asian-Pac-Islander', 'Other',
       'Amer-Indian-Eskimo'], dtype=object)

In [18]:
# Пол. Также все хорошо
data['gender'].unique()

array(['Male', 'Female'], dtype=object)

In [19]:
# Заменяю Male and Female на 0 и 1
data['gender'] = data['gender'].replace(['Male','Female'], [0,1])

In [20]:
# Прирост капитала. Также все хорошо
data['capital-gain'].unique()

array([    0,  7688,  3103,  6418,  7298,  3908, 14084,  5178, 15024,
       99999,  2597,  2907,  4650,  6497,  5013, 27828,  4934,  4064,
        3674,  2174, 10605,  1055,   114,  2580,  3411,  4508,  4386,
        8614, 13550,  6849,  2463,  3137,  2885,  2964,  1471, 10566,
        2354,  1424,  1455,  3325,  4416, 25236,  4787,  2829,  4865,
        1264,  1506,   594, 10520,  3464, 20051,  4101,  1797,  2407,
        3471,  2653,  1086,  1848, 14344,  1151,  2993,  2290, 15020,
        9386,  2202,  3818,  2176,  5455,  3418, 11678,  7978,  7262,
        6514, 41310,  3456,  7430,  2062, 34095,  1831,  6723,  5060,
       15831,  2977,  2346,  3273,  2329,  9562,  2105,  2635,  2414,
        4931,  1731,  6097,   914,  7896,  5556,  3781,  3942,  2538,
        3887,  1409, 25124,  7443,  5721,   401,  1173,  4687,  6767,
        2961,   991,  2036,  2050,  2228, 22040,  2936,  6360,  2009,
        3432,  1639, 18481,  2387], dtype=int64)

In [21]:
# Потеря капитала. Также все хорошо
data['capital-loss'].unique()

array([   0, 1721, 1876, 2415, 1887,  625, 1977, 2057, 1429, 1590, 1485,
       2051, 2377, 1672, 1628, 1902, 1741, 2444, 1408, 2001, 2042, 1740,
       1825, 1848, 1719, 3004, 2179, 1573, 2205, 1258, 2339, 1726, 2258,
       1340, 1504, 2559, 1668, 1974, 1980, 1564, 2547, 2002, 1669, 1617,
        323, 1602, 3175, 2472, 1579, 2129, 1735, 2282, 1870, 1911, 1092,
       1762, 2457, 2231, 2238,  653, 1138, 2246, 2603, 2392, 1510, 1944,
       1380, 1421, 3770, 1594,  213, 2824, 2174, 1844, 2149, 1411, 2467,
       2163, 1816, 1651, 2206, 2352, 1539,  880,  810,  974, 1648, 1755,
       3683, 4356, 2267, 2080,  419,  155, 3900, 2201, 2754], dtype=int64)

In [22]:
# Количество рабочих часов в неделю. Все хорошо, нет пропущенных (непонятных) значений
data['hours-per-week'].unique()

array([40, 50, 30, 32, 10, 39, 48, 25, 20, 45, 47, 35, 43, 90, 54, 60, 38,
       36, 18, 24, 44, 56, 28, 16, 41, 22, 55, 14, 33, 37, 12, 70, 15, 75,
       52, 84, 42, 80,  8, 68, 99, 65, 17, 72, 53, 96, 21, 46,  3,  1, 49,
       67, 76, 23,  2, 58, 26, 34,  4,  7, 51, 78, 63,  5, 31, 92, 77, 27,
       29, 85,  6, 13, 19, 98, 62, 66, 57, 86, 11, 59,  9, 64, 73, 61, 88,
       79, 89, 74, 69, 87, 97, 94, 82, 91, 81, 95], dtype=int64)

In [23]:
# Страна. Есть пропущенные значения.
data['native-country'].unique()

array(['United-States', '?', 'Peru', 'Guatemala', 'Mexico',
       'Dominican-Republic', 'Ireland', 'Germany', 'Philippines',
       'Thailand', 'Haiti', 'El-Salvador', 'Puerto-Rico', 'Vietnam',
       'South', 'Columbia', 'Japan', 'India', 'Cambodia', 'Poland',
       'Laos', 'England', 'Cuba', 'Taiwan', 'Italy', 'Canada', 'Portugal',
       'China', 'Nicaragua', 'Honduras', 'Iran', 'Scotland', 'Jamaica',
       'Ecuador', 'Yugoslavia', 'Hungary', 'Hong', 'Greece',
       'Trinadad&Tobago', 'Outlying-US(Guam-USVI-etc)', 'France',
       'Holand-Netherlands'], dtype=object)

In [24]:
# Есть пропущенные значения: '?'
# смотрим сколько таких строк:
data['native-country'].value_counts()

United-States                 41292
Mexico                          903
?                               811
Philippines                     283
Germany                         193
Puerto-Rico                     175
Canada                          163
India                           147
El-Salvador                     147
Cuba                            133
England                         119
China                           113
Jamaica                         103
South                           101
Italy                           100
Dominican-Republic               97
Japan                            89
Guatemala                        86
Vietnam                          83
Columbia                         82
Poland                           81
Haiti                            69
Portugal                         62
Iran                             56
Taiwan                           55
Greece                           49
Nicaragua                        48
Peru                        

In [25]:
# Таких 811 строк. Удаляю их:
data = data.loc[data['native-country'] != '?']
data['native-country'].value_counts()

United-States                 41292
Mexico                          903
Philippines                     283
Germany                         193
Puerto-Rico                     175
Canada                          163
El-Salvador                     147
India                           147
Cuba                            133
England                         119
China                           113
Jamaica                         103
South                           101
Italy                           100
Dominican-Republic               97
Japan                            89
Guatemala                        86
Vietnam                          83
Columbia                         82
Poland                           81
Haiti                            69
Portugal                         62
Iran                             56
Taiwan                           55
Greece                           49
Nicaragua                        48
Peru                             45
Ecuador                     

In [26]:
# Уровенб ЗП. Тут все ок.
data['income'].unique()

array(['<=50K', '>50K'], dtype=object)

In [27]:
# Осталось строк
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              45222 non-null  int64 
 1   workclass        45222 non-null  object
 2   fnlwgt           45222 non-null  int64 
 3   education        45222 non-null  object
 4   educational-num  45222 non-null  int64 
 5   marital-status   45222 non-null  object
 6   occupation       45222 non-null  object
 7   relationship     45222 non-null  object
 8   race             45222 non-null  object
 9   gender           45222 non-null  int64 
 10  capital-gain     45222 non-null  int64 
 11  capital-loss     45222 non-null  int64 
 12  hours-per-week   45222 non-null  int64 
 13  native-country   45222 non-null  object
 14  income           45222 non-null  object
dtypes: int64(7), object(8)
memory usage: 5.5+ MB


In [28]:
print('Всего строк в наборе {}'.format( len( data ) ))

Всего строк в наборе 45222


In [29]:
# выбираю признаки:
selectedColumns = data[['age', 'workclass', 'education', 'educational-num', 'marital-status', 'occupation', \
                        'relationship', 'race', 'gender', 'capital-gain', 'capital-loss', 'hours-per-week', \
                        'native-country']]

# столбец является категориальной переменной
# переведем ее в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies(selectedColumns, columns = ['workclass', 'education', 'educational-num', 'marital-status', \
                                               'occupation', 'relationship', 'race', 'native-country'])


X.head()

,age,gender,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,0,0,0,40,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,0,0,0,50,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,0,0,0,40,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44,0,7688,0,40,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,0,0,0,30,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [32]:
# перевежу значения столбца в числа, оставив один столбец
le.fit(data['income'])

LabelEncoder()

In [33]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [34]:
# записываю в переменную y преобразованный столбец income

y = pd.Series( data = le.transform( data['income'] ) )
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

In [35]:
model = LogisticRegression(max_iter=2000)

In [36]:
X

,age,gender,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,0,0,0,40,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,0,0,0,50,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,0,0,0,40,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44,0,7688,0,40,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,0,0,0,30,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,1,0,0,38,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
48838,40,0,0,0,40,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
48839,58,1,0,0,40,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
48840,22,0,0,0,20,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [37]:
# обучаем модель
model.fit( X, y )

LogisticRegression(max_iter=2000)

In [38]:
predictions = model.predict_proba( X )

In [39]:
predictions[:5]

array([[0.99768511, 0.00231489],
       [0.86636713, 0.13363287],
       [0.62765143, 0.37234857],
       [0.25257864, 0.74742136],
       [0.99654108, 0.00345892]])

In [40]:
# сравниваю факт с предсказаниями
# (сравнивать предсказания с самими данными не гуд, лучше разбить на обучающую и тестовую выборку, смотри var 2)

zip( predictions[:, 1], y )

In [41]:
model.predict(X)


array([0, 0, 0, ..., 0, 0, 1])

In [42]:
model.score(X, y)

0.8480385653000752